In [12]:
import pandas as pd
import numpy as np

from sklearn.metrics import confusion_matrix
from pathlib import Path

In [30]:
# set the paths for the intersectional analysis
data_path = Path("./data/intersectional_test.csv")
embed_path = Path("./results/noise_results/interseks_eval\_test.csv")

In [31]:
# read the data
df_orig = pd.read_csv(data_path)
df_embed = pd.read_csv(embed_path)

# reverse the dummies for the sensitive attribute
s_columns = [col for col in df_embed.columns if 's_' in col]
df_orig["s_pred"] = pd.from_dummies(df_embed[s_columns], sep="_")

# reverse the dummies for the target variable
y_columns = [col for col in df_embed.columns if 'y_' in col]
df_orig["y_pred"] = pd.from_dummies(df_embed[y_columns], sep="_").astype(int)

# get the embeddings
embed_columns = [col for col in df_embed.columns if 'h_' in col]
df_orig["representation"] = df_embed[embed_columns].values.tolist()

# get an overview of the dataframe
df_orig.head()

,limit_bal,education,age,pay_0,pay_2,pay_3,pay_4,pay_5,pay_6,bill_amt1,...,pay_amt2,pay_amt3,pay_amt4,pay_amt5,pay_amt6,y,intersection,s_pred,y_pred,representation
0,130000,2,37,0,0,0,0,0,0,33346,...,5542,4500,5465,4500,5662,0,21,3,0,"[0.2304390221834182, 0.0018366500735282, 0.121..."
1,210000,1,28,0,0,0,0,0,0,124667,...,5065,5012,5000,5078,5000,0,11,3,0,"[0.1967833787202835, 0.0026455377228558, 0.086..."
2,30000,2,38,1,2,0,0,0,2,24305,...,1410,1423,2051,0,2000,1,11,3,1,"[0.1154541447758674, 0.0012526481878012, 0.177..."
3,180000,3,28,0,0,0,0,0,0,113125,...,1501,1000,0,0,49347,0,21,3,0,"[0.1509087532758712, 0.003601809265092, 0.1246..."
4,80000,3,45,1,-1,0,0,-1,-1,-1213,...,3000,0,390,390,1170,0,11,3,0,"[0.1476174592971801, 0.0094154747202992, 0.142..."


In [32]:
def get_metrics(y_true, y_pred, threshold=0.5):
    """
    Threshold is used to convert probabilities to binary predictions.
    If threshold is None, then y_pred is assumed to be binary predictions already.
    """
    if threshold is not None:  # Check if threshold is passed as arg
        y_pred = (y_pred > threshold).astype(int)

    # Calculate different evaluation metrics
    confusion_mat = confusion_matrix(y_true, y_pred)

    # Since 0 is the prefered class (not defaulting), we flip the confusion matrix
    # Positive class: 0, did not default
    # Negative class: 1, defaulted
    TP = confusion_mat[0, 0]
    FN = confusion_mat[0, 1]
    FP = confusion_mat[1, 0]
    TN = confusion_mat[1, 1]

    # Statistical Parity
    SP = (TP + FP) / (TP + FP + TN + FN)  # P / N

    # Equal Opportunity, same as True Positive Rate (TPR), or Recall
    EO = TP / (TP + FN)

    TNR = TN / (TN + FP)  # True Negative Rate
    FPR = FP / (FP + TN)  # False Positive Rate
    FNR = FN / (FN + TP)  # False Negative Rate

    accuracy = TP / (TP + TN + FP + FN)
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    f1 = 2 * (precision * recall) / (precision + recall)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1, "matrix": confusion_mat,
            "TN": TN, "FP": FP, "FN": FN, "TP": TP, "SP": SP, "EO": EO, "TNR": TNR, "FPR": FPR, "FNR": FNR}

In [33]:
labels_intersection = {11: "Married Men", 12: "Single Men", 21: "Married Women", 22: "Single Women"}

# get the metrics
subgroups = sorted(df_orig['intersection'].unique())
subgroup_metrics = []

for subgroup in subgroups:
    df = df_orig[df_orig['intersection'] == subgroup]
    
    # get the metrics
    metrics = get_metrics(df['y'], df['y_pred'], threshold=None)
    specs = [metrics["f1"], metrics["SP"], metrics["EO"]]
    subgroup_metrics.append(specs)

    # print the stuff
    print(f'{labels_intersection.get(subgroup)};  f1:{specs[0]}, sp:{specs[1]}, eo:{specs[2]}')

Married Men;  f1:0.8625204582651391, sp:0.8161953727506427, eo:0.8977853492333902
Single Men;  f1:0.8834278512917454, sp:0.8351983723296033, eo:0.9151436031331592
Married Women;  f1:0.8819815444390482, sp:0.8497246262785209, eo:0.9274770173646578
Single Women;  f1:0.8976784178847808, sp:0.839943342776204, eo:0.9157894736842105


In [23]:
# get the mean and std's
print("mean -> f1:{0}, sp: {1}, eo: {2}".format(*np.mean(subgroup_metrics, axis=0)))
print("std -> f1:{0}, sp: {1}, eo: {2}".format(*np.std(subgroup_metrics, axis=0)))

mean -> f1:0.8814020679701784, sp: 0.8352654285337426, eo: 0.9140488608538544
std -> f1:0.01250874037063607, sp: 0.01219244443484857, eo: 0.010595345572359895


## check for the original dataset

In [24]:
# set the paths for the orig dataset analysis
data_path = Path("./data/orig_test.csv")
embed_path = Path("./results/noise_results/orig_eval/_test.csv")

In [26]:
# read the data
df_orig = pd.read_csv(data_path)
df_embed = pd.read_csv(embed_path)

# reverse the dummies for the sensitive attribute
s_columns = [col for col in df_embed.columns if 's_' in col]
df_orig["s_pred"] = pd.from_dummies(df_embed[s_columns], sep="_")

# reverse the dummies for the target variable
y_columns = [col for col in df_embed.columns if 'y_' in col]
df_orig["y_pred"] = pd.from_dummies(df_embed[y_columns], sep="_").astype(int)

# get the embeddings
embed_columns = [col for col in df_embed.columns if 'h_' in col]
df_orig["representation"] = df_embed[embed_columns].values.tolist()

# get an overview of the dataframe
df_orig.head()

,limit_bal,sex,education,marriage,age,pay_0,pay_2,pay_3,pay_4,pay_5,...,pay_amt1,pay_amt2,pay_amt3,pay_amt4,pay_amt5,pay_amt6,y,s_pred,y_pred,representation
0,130000,2,2,1,37,0,0,0,0,0,...,5448,5542,4500,5465,4500,5662,0,1,0,"[0.0810558274388313, 0.0075113670900464, 0.031..."
1,210000,1,1,1,28,0,0,0,0,0,...,145273,5065,5012,5000,5078,5000,0,1,0,"[0.1637899279594421, 0.0978748574852943, 0.002..."
2,30000,1,2,1,38,1,2,0,0,0,...,0,1410,1423,2051,0,2000,1,1,1,"[0.0439501032233238, 0.0026734496932476, 0.058..."
3,180000,2,3,1,28,0,0,0,0,0,...,1923,1501,1000,0,0,49347,0,1,0,"[0.08223457634449, 0.0076815686188638, 0.08325..."
4,80000,1,3,1,45,1,-1,0,0,-1,...,38677,3000,0,390,390,1170,0,1,0,"[0.0750195458531379, 0.0050950129516422, 0.074..."


In [29]:
# get the metrics
subgroups = sorted(df_orig['sex'].unique())
subgroup_metrics = []

for subgroup in subgroups:
    df = df_orig[df_orig['sex'] == subgroup]
    
    # get the metrics
    metrics = get_metrics(df['y'], df['y_pred'], threshold=None)
    specs = [metrics["f1"], metrics["SP"], metrics["EO"]]
    subgroup_metrics.append(specs)

    # print the stuff
    print(f'{subgroup};  f1:{specs[0]}, sp:{specs[1]}, eo:{specs[2]}')

1;  f1:0.8763326226012793, sp:0.8296422487223168, eo:0.9113082039911308
2;  f1:0.8883826879271072, sp:0.8464405516213194, eo:0.9202453987730062
